In [ ]:
using Logging
using Turing
using PyCall
mpl = pyimport_conda("matplotlib", "matplotlib")
mpl[:use]("TkAgg")
using PyPlot
using Nullables
using Distributions
using Printf: @sprintf
import Random

disable_logging(Logging.Info)

# Scenes and path planning (same as Gen)

In [ ]:
#############
# 2D scenes #
#############

struct Point
    x::Float64
    y::Float64
end

Point(arr::Vector{Float64}) = Point(arr[1], arr[2])

array(point::Point) = [point.x, point.y]

function dist(a::Point, b::Point)
    dx = a.x - b.x
    dy = a.y - b.y
    sqrt(dx * dx + dy * dy)
end


Base.:+(a::Point, b::Point) = Point(a.x + b.x, a.y + b.y)
Base.:*(a::Real, b::Point) = Point(b.x * a, b.y * a)


abstract type Obstacle end

"""
A 2D canvas with a set of obstacles
"""
mutable struct Scene
    xmin::Float64
    xmax::Float64
    ymin::Float64
    ymax::Float64
    obstacles::Array{Obstacle,1}
end

function Scene(xmin::Real, xmax::Real, ymin::Real, ymax::Real)
    Scene(xmin, xmax, ymin, ymax, Obstacle[])
end

function add!(scene::Scene, object::Obstacle)
    push!(scene.obstacles, object)
end

function ccw(a::Point, b::Point, c::Point)
    (b.x - a.x) * (c.y - a.y) - (c.x - a.x) * (b.y - a.y)
end

function line_intersects_line(a1::Point, a2::Point, b1::Point, b2::Point)
        # http://algs4.cs.princeton.edu/91primitives/
        if ccw(a1, a2, b1) * ccw(a1, a2, b2) > 0
            return false
        end
        if ccw(b1, b2, a1) * ccw(b1, b2, a2) > 0
            return false
        end
        return true
end

function line_of_site(scene::Scene, a::Point, b::Point)
    for obstacle in scene.obstacles
        if intersects_path(obstacle, a, b)
            return false
        end
    end
    return true
end

"""
An arbitrary polygon
"""
struct Polygon <: Obstacle
    vertices::Array{Point,1}
end

function intersects_path(poly::Polygon, path_start::Point, path_end::Point)
    n = length(poly.vertices)
    for start_vertex_idx=1:n
        end_vertex_idx = start_vertex_idx % n + 1 # loop over to 1
        v1 = poly.vertices[start_vertex_idx]
        v2 = poly.vertices[end_vertex_idx]
        if line_intersects_line(v1, v2, path_start, path_end)
            return true
        end
    end
    return false
end

"""
A line with thickness
"""
struct Wall <: Obstacle
    start::Point
    orientation::Int # x is 1, y is 2
    length::Float64
    thickness::Float64
    height::Float64
    poly::Polygon
    function Wall(start::Point, orientation::Int, length::Float64,
                  thickness::Float64, height::Float64)
        if orientation != 1 && orientation != 2
            error("orientation must be either 1 (x) or 2 (y)")
        end
        vertices = Vector{Point}(undef, 4)
        vertices[1] = start
        dx = orientation == 1 ? length : thickness
        dy = orientation == 2 ? length : thickness
        vertices[2] = Point(start.x + dx, start.y)
        vertices[3] = Point(start.x + dx, start.y + dy)
        vertices[4] = Point(start.x, start.y + dy)
        poly = Polygon(vertices)
        new(start, orientation, length, thickness, height, poly)
    end
end

function intersects_path(wall::Wall, path_start::Point, path_end::Point)
    intersects_path(wall.poly, path_start, path_end)
end

"""
A square
"""
struct Tree <: Obstacle
    center::Point
    size::Float64
    poly::Polygon
    function Tree(center::Point; size::Float64=10.)
        vertices = Vector{Point}(undef, 4)
        vertices[1] = Point(center.x - size/2, center.y - size/2)
        vertices[2] = Point(center.x + size/2, center.y - size/2)
        vertices[3] = Point(center.x + size/2, center.y + size/2)
        vertices[4] = Point(center.x - size/2, center.y + size/2)
        poly = Polygon(vertices)
        new(center, size, poly)
    end
end

function intersects_path(tree::Tree, path_start::Point, path_end::Point)
    intersects_path(tree.poly, path_start, path_end)
end


##########
# render #
##########

const patches = PyPlot.matplotlib[:patches]

function render(polygon::Polygon, ax)
    num_sides = length(polygon.vertices)
    vertices = Matrix{Float64}(undef, num_sides, 2)
    for (i, pt) in enumerate(polygon.vertices)
        vertices[i,:] = [pt.x, pt.y]
    end
    poly = patches[:Polygon](vertices, true, facecolor="black")
    ax[:add_patch](poly)
end

render(wall::Wall, ax) = render(wall.poly, ax)
render(tree::Tree, ax) = render(tree.poly, ax)

function render(scene::Scene, ax)
    ax[:set_xlim]((scene.xmin, scene.xmax))
    ax[:set_ylim]((scene.ymin, scene.ymax))
    for obstacle in scene.obstacles
        render(obstacle, ax)
    end
end

In [ ]:
# depends on: scenes.jl

###############
# Generic RRT #
###############

struct RRTNode{C,U}
    conf::C
    # the previous configuration and the control needed to produce this node's
    # configuration from the previous configuration may be null if it is the
    # root node
    parent::Nullable{RRTNode{C,U}}
    control::Nullable{U}
    cost_from_start::Float64 # cost (e.g. distance) of moving from start to us
end

mutable struct RRTTree{C,U}
    nodes::Array{RRTNode{C,U}, 1}
end

function RRTTree(root_conf::C, ::Type{U}) where {C,U}
    nodes = Array{RRTNode{C,U},1}()
    push!(nodes, RRTNode(root_conf,
        Nullable{RRTNode{C,U}}(), Nullable{U}(), 0.0))
    RRTTree(nodes)
end


function Base.show(io::IO, tree::RRTTree{C,U}) where {C,U}
    write(io, "RRTTree with $(length(tree.nodes)) nodes")
end

function add_node!(tree::RRTTree{C,U}, parent::RRTNode{C,U},
                   new_conf::C, control::U,
                   cost_from_start::Float64) where {C,U}
    node = RRTNode(new_conf,
        Nullable{RRTNode{C,U}}(parent), Nullable{U}(control),
        cost_from_start)
    push!(tree.nodes, node)
    return node
end

root(tree::RRTTree{C,U}) where {C,U} = tree.nodes[1]

abstract type RRTScheme{C,U} end

function nearest_neighbor(scheme::RRTScheme{C,U}, conf::C,
                               tree::RRTTree{C,U}) where {C,U}
    nearest::RRTNode{C,U} = root(tree)
    best_dist::Float64 = dist(nearest.conf, conf)
    for node::RRTNode{C,U} in tree.nodes
        d = dist(node.conf, conf)
        if d < best_dist
            best_dist = d
            nearest = node
        end
    end
    nearest
end

struct SelectControlResult{C,U}
    start_conf::C
    new_conf::C
    control::U
    failed::Bool # new_conf is undefined in this case
    cost::Float64 # cost of this control action (e.g. distance)
end

function rrt(scheme::RRTScheme{C,U}, init::C, iters::Int, dt::Float64) where {C,U}
    tree = RRTTree(init, U) # init is the root of tree
    for iter=1:iters
        rand_conf::C = random_config(scheme)
        near_node::RRTNode{C,U} = nearest_neighbor(scheme, rand_conf, tree)
        result = select_control(scheme, rand_conf, near_node.conf, dt)
        if !result.failed
            cost_from_start = near_node.cost_from_start + result.cost
            add_node!(tree, near_node, result.new_conf, result.control, cost_from_start)
        end
    end
    tree
end


##############################
# RRT for holonomic 2D point #
##############################

struct HolonomicPointRRTScheme <: RRTScheme{Point,Point}
    scene::Scene
end

function random_config(scheme::HolonomicPointRRTScheme)
    x = rand() * (scheme.scene.xmax - scheme.scene.xmin) + scheme.scene.xmin
    y = rand() * (scheme.scene.ymax - scheme.scene.ymin) + scheme.scene.ymin
    Point(x, y)
end

function select_control(scheme::HolonomicPointRRTScheme, 
                        target_conf::Point, start_conf::Point, dt::Float64)

    dist_to_target = dist(start_conf, target_conf)
    diff = Point(target_conf.x - start_conf.x, target_conf.y - start_conf.y)
    distance_to_move = min(dt, dist_to_target)
    scale = distance_to_move / dist_to_target
    control = Point(scale * diff.x, scale * diff.y)

    obstacles = scheme.scene.obstacles

    # go in the direction of target_conf from start_conf 
    new_conf = Point(start_conf.x + control.x, start_conf.y + control.y)

    # test the obstacles
    failed = false
    for obstacle in obstacles
        if intersects_path(obstacle, start_conf, new_conf)
            # NOTE: could do more intelligent things like backtrack until you succeed
            failed = true
            break
        end
    end
    cost = distance_to_move
    SelectControlResult(start_conf, new_conf, control, failed, cost)
end


################
# path planner #
################

struct PlannerParams
    rrt_iters::Int
    rrt_dt::Float64 # the maximum proposal distance
    refine_iters::Int
    refine_std::Float64
end

struct Path
    start::Point
    goal::Point
    points::Array{Point,1}
end

function concatenate(a::Path, b::Path)
    if a.goal.x != b.start.x || a.goal.y != b.start.y
        error("goal of first path muts be start of second path")
    end
    points = Array{Point,1}()
    for point in a.points
        push!(points, point)
    end
    for point in b.points[2:end]
        push!(points, point)
    end
    @assert points[1].x == a.start.x
    @assert points[1].y == a.start.y
    @assert points[end].x == b.goal.x
    @assert points[end].y == b.goal.y
    Path(a.start, b.goal, points)
end

"""
Remove intermediate nodes that are not necessary, so that refinement
optimization is a lower-dimensional optimization problem.
"""
function simplify_path(scene::Scene, original::Path)
    new_points = Array{Point,1}()
    push!(new_points, original.start)
    for i=2:length(original.points) - 1
        if !line_of_site(scene, new_points[end], original.points[i + 1])
            push!(new_points, original.points[i])
        end
    end
    @assert line_of_site(scene, new_points[end], original.goal)
    push!(new_points, original.goal)
    Path(original.start, original.goal, new_points)
end

"""
Optimize the path to minimize its length while avoiding obstacles in the scene.
"""
function refine_path(scene::Scene, original::Path, iters::Int, std::Float64)
    # do stochastic optimization
    new_points = deepcopy(original.points)
    num_interior_points = length(original.points) -2
    if num_interior_points == 0
        return original
    end
    for i=1:iters
        point_idx = 2 + (i % num_interior_points)
        @assert point_idx > 1 # not start
        @assert point_idx < length(original.points) # not goal
        prev_point = new_points[point_idx-1]
        point = new_points[point_idx]
        next_point = new_points[point_idx+1]
        adjusted = Point(point.x + randn() * std, point.y + randn() * std)
        cur_dist = dist(prev_point, point) + dist(point, next_point)
        ok_backward = line_of_site(scene, prev_point, adjusted)
        ok_forward = line_of_site(scene, adjusted, next_point)
        if ok_backward && ok_forward
            new_dist = dist(prev_point, adjusted) + dist(adjusted, next_point)
            if new_dist < cur_dist 
                # accept the change
                new_points[point_idx] = adjusted
            end
        end
    end
    Path(original.start, original.goal, new_points)
end

function optimize_path(scene::Scene, original::Path, refine_iters::Int, refine_std::Float64)
    #simplified = simplify_path(scene, original) # TODO?
    refined = refine_path(scene, original, refine_iters, refine_std)
    refined
end

"""
Plan path from start to goal that avoids obstacles in the scene.
"""
function plan_path(start::Point, goal::Point, scene::Scene,
                   params::PlannerParams=PlannerParams(2000, 3.0, 10000, 1.))
    scheme = HolonomicPointRRTScheme(scene)
    tree = rrt(scheme, start, params.rrt_iters, params.rrt_dt)

    # find the best path along the tree to the goal, if one exists
    best_node = tree.nodes[1]
    min_cost = Inf
    path_found = false
    for node in tree.nodes
        # check for line-of-site to the goal
        clear_path = line_of_site(scene, node.conf, goal)
        cost = node.cost_from_start + (clear_path ? dist(node.conf, goal) : Inf)
        if cost < min_cost
            path_found = true
            best_node = node
            min_cost = cost
        end
    end

    local path::Nullable{Path}
    if path_found
        # extend the tree to the goal configuration
        control = Point(goal.x - best_node.conf.x, goal.y - best_node.conf.y)
        goal_node = add_node!(tree, best_node, goal, control, min_cost)
        points = Array{Point,1}()
        node::RRTNode{Point,Point} = goal_node
        push!(points, node.conf)
        # the path will contain the start and goal
        while !isnull(node.parent)
            node = get(node.parent)
            push!(points, node.conf)
        end
        @assert points[end] == start # the start point
        @assert points[1] == goal
        path = Nullable{Path}(Path(start, goal, reverse(points)))
    else
        path = Nullable{Path}()
    end
    
    local optimized_path::Nullable{Path}
    if path_found
        optimized_path = Nullable{Path}(optimize_path(scene, get(path),
                                                      params.refine_iters, params.refine_std))
    else
        optimized_path = Nullable{Path}()
    end
    return optimized_path
end

"""
Sample the location of a walk along a path at given time points, for a fixed speed.
"""
function walk_path(path::Path, speed::Float64, times::Array{Float64,1})
    distances_from_start = Vector{Float64}(undef, length(path.points))
    distances_from_start[1] = 0.0
    for i=2:length(path.points)
        distances_from_start[i] = distances_from_start[i-1] + dist(path.points[i-1], path.points[i])
    end
    locations = Vector{Point}(undef, length(times))
    locations[1] = path.points[1]
    for (time_idx, t) in enumerate(times)
        if t < 0.0
            error("times must be positive")
        end
        desired_distance = t * speed
        used_up_time = false
        # NOTE: can be improved (iterate through path points along with times)
        for i=2:length(path.points)
            prev = path.points[i-1]
            cur = path.points[i]
            dist_to_prev = dist(prev, cur)
            if distances_from_start[i] >= desired_distance
                # we overshot, the location is between i-1 and i
                overshoot = distances_from_start[i] - desired_distance
                @assert overshoot <= dist_to_prev
                past_prev = dist_to_prev - overshoot
                frac = past_prev / dist_to_prev
                locations[time_idx] = Point(prev.x * (1. - frac) + cur.x * frac,
                                     prev.y * (1. - frac) + cur.y * frac)
                used_up_time = true
                break
            end
        end
        if !used_up_time
            # sit at the goal indefinitely
            locations[time_idx] = path.goal
        end
    end
    locations
end

# Turing model

In [ ]:
@model path_example(xs, ys, start, scene, times) = begin
    start[1] ~ Uniform(0,1)
    start[2] ~ Uniform(0,1)
    start_pt = Point(start[1], start[2])
    
    stop_x ~ Uniform(0, 1)
    stop_y ~ Uniform(0, 1)
    stop_pt = Point(stop_x, stop_y)
    
    maybe_path = plan_path(start_pt, stop_pt, scene, PlannerParams(300, 3.0, 2000, 1.))
    
    speed ~ Uniform(0, 1)
    
    if !isnull(maybe_path)
        path = get(maybe_path)
        locations = walk_path(path, speed, times)
    else
        locations = fill(start_pt, length(times))
    end
    
    noise_var ~ Uniform(0, 1)
    noise = noise_var * 0.1
        
    for (i, point) in enumerate(locations)
        xs[i] ~ Normal(point.x, noise)
        ys[i] ~ Normal(point.y, noise)
    end
    
    (start, stop, speed, noise, maybe_path, locations)
end

# Making the data (the scene, the times, the ground truth path)

In [ ]:
function make_scene()
    scene = Scene(0, 1, 0, 1) 
    add!(scene, Tree(Point(0.30, 0.20), size=0.1))
    add!(scene, Tree(Point(0.83, 0.80), size=0.1))
    add!(scene, Tree(Point(0.80, 0.40), size=0.1))
    horiz = 1
    vert = 2
    wall_height = 0.30
    wall_thickness = 0.02
    walls = [
        Wall(Point(0.20, 0.40), horiz, 0.40, wall_thickness, wall_height)
        Wall(Point(0.60, 0.40), vert, 0.40, wall_thickness, wall_height)
        Wall(Point(0.60 - 0.15, 0.80), horiz, 0.15 + wall_thickness, wall_thickness, wall_height)
        Wall(Point(0.20, 0.80), horiz, 0.15, wall_thickness, wall_height)
        Wall(Point(0.20, 0.40), vert, 0.40, wall_thickness, wall_height)]
    for wall in walls
        add!(scene, wall)
    end
    return scene
end

const scene = make_scene()
const times = collect(range(0, stop=1, length=20))

In [ ]:
function make_ground_truth(scene, times)
    start_x = 0.1
    start_y = 0.1
    stop_x = 0.5
    stop_y = 0.5
    noise = 0.01
    start = Point(start_x, start_y)
    stop = Point(stop_x, stop_y)
    maybe_path = plan_path(start, stop, scene, PlannerParams(500, 3.0, 2000, 1.))
    speed = rand(Uniform(0,1))
    
    if !isnull(maybe_path)
        path = get(maybe_path)
        locations = walk_path(path, speed, times)
    else
        locations = fill(start, length(times))
    end
    
    obs = map(p -> Point(rand(Normal(p.x, noise)), rand(Normal(p.y, noise))), locations)
    (start, stop, speed, noise, maybe_path, locations, obs)
end

In [ ]:
function render(scene::Scene, nums, ax; num_measurements=20,
                show_measurements=true,
                show_start=true, show_stop=true,
                show_path=true, show_noise=true,
                start_alpha=1., stop_alpha=1., path_alpha=1.)

    start = Point(0.1, 0.1)
    stop_x, speed, _, _, stop_y = nums
    stop = Point(stop_x, stop_y)

    render(scene, ax)
    sca(ax)
    if show_start
        scatter([start.x], [start.y], color="blue", s=100, alpha=start_alpha)
    end
    if show_stop
        scatter([stop.x], [stop.y], color="red", s=100, alpha=stop_alpha)
    end
    
    # plot measured locations
    if show_measurements
        scatter(xs[1:num_measurements], ys[1:num_measurements], marker="x", color="black", alpha=1., s=25)
    end
end

In [ ]:
# We need to generate some data to pass in.
(start, stop, speed, noise, maybe_path, locations, obs) = make_ground_truth(scene,times)
xs = map(p -> p.x, obs)
ys = map(p -> p.y, obs)

figure(figsize=(4,4))
ax = gca()
render(scene, (stop.x, speed, 0, 0, stop.y, noise), ax)
savefig("ground_truth_turing.png")

In [ ]:
# For each number of observations, generate 20 traces.
iters_per_chain = 100
for i=1:20
    println("Observing $(i) steps...")
    figure(figsize=(4,4))
    ax=gca()
    for j=1:20
        @time chain = sample(path_example(xs[1:i], ys[1:i], [0.1, 0.1], scene, times[1:i]), Gibbs(iters_per_chain, MH(1, :noise_var), MH(1, :speed), MH(1, :stop_x, :stop_y)))
        nums = chain.value[iters_per_chain,:,:]
        render(scene,nums,ax,stop_alpha=0.2,path_alpha=0.5,num_measurements=i)
    end
    fname = "turing_inferred_$(i)"
    savefig(fname)
end